### Commands

In [23]:
from firecloud import api as firecloud_api
import glob
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

In [336]:
### Prepare cohorts for export to FC
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [358]:
# # Delete cohorts
# cohort_codes = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None)
# for coh_code in cohort_codes.loc[:, 1].tolist():
#     res = fc_interface.delete_sample_set(namespace, workspace, coh_code)

In [338]:
# ### Upload metadata for given batch
# fc_interface.update_batch_metadata('TSCA21', \
#                                   '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
#                                       paths_to_samples_info, \
#                                         namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [361]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [359]:
# Cohorts
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [66]:
### Create panel of normals from batch
pon = fc_interface.create_panel_of_normals_from_batch(batch_id, paths_to_samples_info, N=20).shape

In [78]:
### Create panel of normals from batch
for i, row in batches_info.iterrows():
    pon, name = fc_interface.create_panel_of_normals_from_batch(row['tsca_id'], paths_to_samples_info, N=20)
    fc_interface.upload_pon(pon, name, namespace, workspace)

In [ ]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [33]:
### Prepare cohorts for export to FC
cohorts = fc_interface.\
            prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, \
                                               blacklist=["CCLF_AA1012-Tumor-SM-F67DF"])

cohorts_sample_set_metadata = "cohort_files/fc_upload_sample_set_cohorts.txt"
res = fc_interface.upload_entities_from_tsv(namespace, workspace, cohorts_sample_set_metadata)

In [8]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [34]:
### Cohort of all tumors
res = fc_interface.upload_cohort_all_tumors(paths_to_samples_info, google_bucket_id, \
                                      'Cum_Tumors_22_all', namespace, workspace, ['DW039-Tumor-SM-DB2IF'])

In [12]:
# fc_interface.merge_walkupseq_files(paths_to_samples_info)

(1045, 14)

---

---

In [ ]:
def update_bsp_data(tsca_id):
    """The walkupseq sheets contain experimental data (collaborator, tumor type, tissue site, etc.).
    Compile all the sheets onto one walkupseq_all_combined_TSCAXX.txt
    Args: 
    - tsca_id: latest TSCA id added
    Writes: 
    - bsp_latest_all_samples_TSCAXX.xls: updated bsp data
    """
    files = glob.glob('/Users/mimoun/production_fc/metadata_exports/walkupseq_files/tsca*')
    dfs = []
    for f in files:
        dfs.append( pd.read_table(f, comment='#', encoding='latin1'))
    combined = pd.concat(dfs, axis=0)
    fname = "/Users/mimoun/production_fc/metadata_exports/walkupseq_files/walkupseq_all_combined_%s.txt"%tsca_id
    combined.to_csv(fname, sep="\t", index=None)
    return combined

## Refined: New Batch Workflow

In [90]:
tsca_id = "TSCA22"
path_to_batch_samples_info = batches_info.loc[batches_info.tsca_id==tsca_id, 'path_to_samples_info'].item()

In [29]:
### ALL SAMPLES
## Prepare
# fc_interface.prepare_patients_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_sample_set_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_samples_for_metadata_export(path_to_batch_samples_info, tsca_id, google_bucket_id)

## Export
# fc_interface.export_batch_metadata_to_fc(tsca_id, namespace, workspace)

In [39]:
### PANEL OF NORMALS BATCH
## Prepare 
# pon, name = fc_interface.create_panel_of_normals_from_batch(tsca_id, paths_to_samples_info, N=20)

# Export
# fc_interface.upload_pon(pon, name, namespace, workspace)

In [13]:
### PANEL OF NORMALS CUMULATIVE
## Prepare
# batches_info = pd.read_excel(paths_to_samples_info)
# PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_22_all')

# Export
# fc_interface.upload_pon(PoN, 'Cum_PoN_22_all', namespace, workspace)

Creating panel of 163 normals


<Response [200]>

In [92]:
### COHORT SAMPLE SETS
## Prepare
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, blacklist, tsca_id)

In [94]:
cohorts['membership:sample_set_id'].value_counts()

COHORT_PEDS         191
COHORT_BT           110
COHORT_CCLF_RCRF     47
COHORT_CSHL_PDAC     47
COHORT_AA            39
COHORT_AB            29
COHORT_CY            25
COHORT_EH            22
COHORT_JL            21
COHORT_JB            18
COHORT_SS            16
COHORT_HG            15
COHORT_NB            14
COHORT_TC            13
COHORT_NH            12
COHORT_EW            11
COHORT_DW             8
COHORT_SP             6
COHORT_CO             6
COHORT_COLO           4
COHORT_JNT            1
COHORT_GG             1
COHORT_PM             1
COHORT_PK             1
Name: membership:sample_set_id, dtype: int64

In [ ]:
### PAIRS